In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization




In [10]:
def load_dataset():
    (trainX, trainY),(testX, testY)= cifar10.load_data()
    trainY=to_categorical(trainY)
    testY= to_categorical(testY)
    return trainX, trainY, testX, testY

    
    

In [11]:
#Function to scale pixels 
def prep_pixels(train, test):
    #convert from integers to floats
    train_norm=train.astype('float32')
    test_norm=test.astype('float32')
    #Normalize to range 0-1
    train_norm=train_norm/255.0
    test_norm=test_norm/255.0
    return train_norm, test_norm

    
    

In [12]:
def define_model():
    model=Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='softmax'))
    opt=SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'] )
    return model

    

In [13]:

def summarize_diagnostics(history):
    pyplot.subplot(211)
    pyplot.title('Cross entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    saved_file= sys.argv[0].split('/')[-1]
    pyplot.savefig(saved_file + '_plot.png')
    pyplot.close()
    

In [14]:
def run_test_harness():
    trainX, trainY, testX, testY= load_dataset()
    trainX, testX=prep_pixels(trainX, testX)
    model=define_model()
    datagen=ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    IT_Train= datagen.flow(trainX, trainY, batch_size=64)
    Steps= int(trainX.shape[0]/64)
    history=model.fit_generator(IT_Train, steps_per_epoch=Steps,epochs=100, validation_data=(testX, testY), verbose=0)
    _, acc=model.evaluate(testX, testY, verbose=0)
    
    print('> %.3f' % (acc * 100.0))
    summarize_diagnostics(history)
    
    
    
    
run_test_harness()
    
        
    
    
    
    
    
    
    
    
    
    
    

/tmp/ipykernel_29/990079124.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(IT_Train, steps_per_epoch=Steps,epochs=100, validation_data=(testX, testY), verbose=0)
2023-07-16 13:29:14.719978: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


> 79.010
